# **Part1: Diffusion Models and Latent Diffusion**



In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # 只使用第3号 GPU

import torch
print("Using GPU:", torch.cuda.get_device_name(0))


Thu Nov 20 19:52:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.169                Driver Version: 570.169        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:21:00.0 Off |                  Off |
| 51%   56C    P0            223W /  450W |    9390MiB /  49140MiB |     52%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Section 1: Overview of Diffusion Models

Let's begin by seeing where diffusion models fit within the zoo of generative models in use today:

![model type diag](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/generative-overview.png)
Overview of different types of generative models. (from [this great blog post](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/))

The core concept behind these models is deceptively simple:
- Take a starting image
- Add some noise, iteratively degrading the image until almost nothing but noise remains.
- Train a model to 'undo' these noise steps.
- To generate, start from pure noise and repeatedly apply the model to 'denoise' our way back to a plausible image.

The model generally has access to some representation of 't' - that is, how many noise steps have been applied to the image. And to make its job easier, we typically get the model to output a prediction for the *noise* that we could subtract from the noisy image to get a less noisy one, rather than spitting out the denoised image.

With this high-level idea in our heads, let's have a go at implementing one!


# Section 2: Implementing One



In [2]:
#@title imports and utility functions
from datasets import load_dataset
from PIL import Image
import torch.nn.functional as F
import os
from tqdm.notebook import tqdm
import torch
import numpy as np


def img_to_tensor(im):
  return torch.tensor(np.array(im.convert('RGB'))/255).permute(2, 0, 1).unsqueeze(0) * 2 - 1

def tensor_to_image(t):
  return Image.fromarray(np.array(((t.squeeze().permute(1, 2, 0)+1)/2).clip(0, 1)*255).astype(np.uint8))

def gather(consts: torch.Tensor, t: torch.Tensor):
    """Gather consts for $t$ and reshape to feature map shape"""
    c = consts.gather(-1, t)
    return c.reshape(-1, 1, 1, 1)

In [3]:
import numpy
import pandas
from datasets import load_dataset

print("numpy:", numpy.__version__)
print("pandas:", pandas.__version__)
print("datasets:", __import__('datasets').__version__)


numpy: 1.23.5
pandas: 2.2.2
datasets: 4.2.0


## 2.1 Dataset

We'll start with a classic small dataset, with 32px square images from 10 classes. For convenience we just pull a version that is avalable on the huggingface hub.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import os
import copy
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from tqdm import tqdm
from datasets import load_dataset
from PIL import Image
from diffusers import UNet2DModel, DDPMScheduler

# ----------------------
# 1. 设置
# ----------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 42
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(SEED)

batch_size = 128
lr = 1e-4
num_workers = 2
num_local_epochs = 2        # 每个客户端每轮训练的 epoch 数
num_global_rounds = 25      # 总共聚合 25 轮
save_dir = "checkpoints_finetune_cifar100"
os.makedirs(save_dir, exist_ok=True)

# ----------------------
# 2. 数据
# ----------------------
normalize_mean = (0.5, 0.5, 0.5)
normalize_std = (0.5, 0.5, 0.5)
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(normalize_mean, normalize_std),
])

train_dataset_raw = load_dataset(
    "parquet",
    data_files=f"data/cifar10/train.parquet",
    split="train"
)
val_dataset_raw = load_dataset(
    "parquet",
    data_files=f"data/cifar10/test.parquet",
    split="train"
)
print(f"训练集样本数: {len(train_dataset_raw)}, 验证集样本数: {len(val_dataset_raw)}")

class CIFARDataset(Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        data = self.dataset[idx]
        image = data.get("img", data.get("image"))
        label = int(data.get("fine_label", data.get("label", 0)))
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        if self.transforms:
            image = self.transforms(image)
        return image, label

train_dataset = CIFARDataset(train_dataset_raw, transforms=transform)
val_dataset = CIFARDataset(val_dataset_raw, transforms=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# ----------------------
# 3. 扩散噪声函数
# ----------------------
n_steps = 1000
beta = torch.linspace(0.0001, 0.04, n_steps).to(device)
alpha = 1. - beta
alpha_bar = torch.cumprod(alpha, dim=0)
def q_xt_x0(x0, t):
    n, c, h, w = x0.shape
    eps = torch.randn_like(x0)
    a_bar = alpha_bar[t].reshape(n, 1, 1, 1)
    noisy = a_bar.sqrt() * x0 + (1 - a_bar).sqrt() * eps
    return noisy, eps

# ----------------------
# 4. 非IID划分
# ----------------------
def create_non_iid_splits(dataset, num_users=10, alpha=0.5):
    # 自动识别标签字段
    if 'fine_label' in dataset.column_names:
        labels = np.array(dataset['fine_label'])
    elif 'label' in dataset.column_names:
        labels = np.array(dataset['label'])
    else:
        raise KeyError("未找到标签列，请检查数据字段名（可能是 'fine_label' 或 'label'）")

    n_classes = np.max(labels) + 1
    user_indices = {i: [] for i in range(num_users)}
    class_indices = [np.where(labels == y)[0] for y in range(n_classes)]

    for idx_list in class_indices:
        np.random.shuffle(idx_list)
        proportions = np.random.dirichlet(np.repeat(alpha, num_users))
        proportions = (np.cumsum(proportions) * len(idx_list)).astype(int)
        splits = [idx_list[proportions[i-1] if i>0 else 0: proportions[i]] for i in range(num_users)]
        for user, split in enumerate(splits):
            user_indices[user].extend(split.tolist())
    return user_indices

num_users = 10
dir_alpha = 0.1
user_data_idx = create_non_iid_splits(train_dataset_raw, num_users, dir_alpha)

# ----------------------
# 5. 加载 Tiny-ImageNet 预训练模型
# ----------------------
model = UNet2DModel(
    sample_size=32,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(32, 64, 128, 256),
    down_block_types=("DownBlock2D", "DownBlock2D", "AttnDownBlock2D", "AttnDownBlock2D"),
    up_block_types=("AttnUpBlock2D", "AttnUpBlock2D", "UpBlock2D", "UpBlock2D"),
).to(device)
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
model.load_state_dict(torch.load("tinyimagenet_ddpm_32/best_model.pt", map_location="cpu"))
print("✅ 模型已成功加载（来自 Tiny-ImageNet 训练）。")

# ----------------------
# 6. 联邦训练与聚合
# ----------------------
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x0, _ in dataloader:
            x0 = x0.to(device)
            t = torch.randint(0, n_steps, (x0.shape[0],), dtype=torch.long).to(device)
            xt, noise = q_xt_x0(x0, t)
            noise_pred = model(xt, t).sample
            loss = nn.functional.mse_loss(noise_pred, noise)
            total_loss += loss.item()
    return total_loss / len(dataloader)

best_val_loss = float("inf")

for round_idx in range(num_global_rounds):
    print(f"\n🌍 Global Round [{round_idx+1}/{num_global_rounds}]")
    user_models = []

    # 每个用户进行本地训练
    for user in tqdm(range(num_users), desc="Users"):
        local_model = copy.deepcopy(model)
        local_model.train()
        optimizer = torch.optim.Adam(local_model.parameters(), lr=lr)

        indices = user_data_idx[user]
        user_dataset = Subset(train_dataset, indices)
        dataloader = DataLoader(user_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

        for epoch in range(num_local_epochs):
            total_loss = 0
            for x0, _ in dataloader:
                x0 = x0.to(device)
                optimizer.zero_grad()
                t = torch.randint(0, n_steps, (x0.shape[0],), dtype=torch.long).to(device)
                xt, noise = q_xt_x0(x0, t)
                noise_pred = local_model(xt, t).sample
                loss = nn.functional.mse_loss(noise_pred, noise)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
        user_models.append(copy.deepcopy(local_model.state_dict()))

    # FedAvg 聚合
    global_dict = copy.deepcopy(model.state_dict())
    for key in global_dict.keys():
        global_dict[key] = torch.stack([user_models[u][key].float() for u in range(num_users)], dim=0).mean(0)
    model.load_state_dict(global_dict)

    # 验证集评估
    val_loss = evaluate(model, val_loader)
    print(f"🔍 Validation Loss after Round {round_idx+1}: {val_loss:.6f}")

    # 保存最优模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"{save_dir}/best_model_{dir_alpha}.pt")
        print(f"💾 New best model saved at round {round_idx+1}, val_loss={val_loss:.6f}")

print(f"\n✅ Training finished. Best validation loss = {best_val_loss:.6f}")


训练集样本数: 50000, 验证集样本数: 10000
✅ 模型已成功加载（来自 Tiny-ImageNet 训练）。


In [6]:
import os
import json
from tqdm import tqdm
import torch
import numpy as np
from torch.utils.data import DataLoader

# ---------- 配置 ----------
save_dir = "checkpoints_finetune_cifar100"
ckpt_path = os.path.join(save_dir, "best_model_0.1.pt") 
model.load_state_dict(torch.load(ckpt_path, map_location=device))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
steps = 100
num_passes = 1
batch_size = 256
num_workers = 4
# ------------------------------------------------



# 输出目录
gen_root = os.path.join(save_dir, f"generated_cifar100_steps_{steps}")
os.makedirs(gen_root, exist_ok=True)


# ---------- 核心：生成并保存到 npy ----------
def generate_dataset_to_npy(dataset, out_prefix, passes=1, steps=250):
    """
    dataset：输入 PyTorch dataset
    out_prefix：例如 generated_cifar10/train
    passes：生成次数
    """

    model.eval()
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False,
                        num_workers=num_workers, drop_last=False)

    total = len(dataset)
    print(f"\nStart generating: total={total}, passes={passes}, steps={steps}")

    # 预申请大数组（HWC 格式保存更适合 numpy）
    sample_shape = (3, 32, 32)    # CIFAR10 默认 shape
    H, W = sample_shape[1], sample_shape[2]

    generated_all = np.zeros((passes, total, H, W, 3), dtype=np.float32)
    real_all      = np.zeros((passes, total, H, W, 3), dtype=np.float32)
    labels_all    = np.zeros((passes, total), dtype=np.int32)

    for p in range(passes):
        print(f"\n=== Pass {p+1}/{passes} ===")

        for batch_idx, (x0, labels) in enumerate(tqdm(loader)):
            x0 = x0.to(device)
            labels = labels.to(device)
            bsz = x0.size(0)

            noise = torch.randn_like(x0).to(device)
            t_tensor = torch.full((bsz,), steps - 1, device=device)
            x = noise_scheduler.add_noise(x0, noise, t_tensor)

            for step in reversed(range(steps)):
                t_tensor = torch.full((bsz,), step, device=device)
                with torch.no_grad():
                    out = model(x, t_tensor)
                    noise_pred = out.sample if hasattr(out, "sample") else out
                    x = noise_scheduler.step(noise_pred, int(step), x).prev_sample

            # [-1,1] → [0,1]
            x_gen = (x.clamp(-1, 1) + 1) / 2.0
            x_real = (x0.clamp(-1, 1) + 1) / 2.0

            # NCHW → NHWC 存储
            x_gen = x_gen.permute(0, 2, 3, 1).cpu().numpy()
            x_real = x_real.permute(0, 2, 3, 1).cpu().numpy()
            labels_np = labels.cpu().numpy()

            start = batch_idx * batch_size
            end = start + bsz

            generated_all[p, start:end] = x_gen
            real_all[p, start:end] = x_real
            labels_all[p, start:end] = labels_np

    # ---- 最终保存 ----
    np.save(f"{out_prefix}_generated.npy", generated_all)
    np.save(f"{out_prefix}_real.npy", real_all)
    np.save(f"{out_prefix}_labels.npy", labels_all)

    print(f"\nSaved npy dataset:\n{out_prefix}_generated.npy\n{out_prefix}_real.npy\n{out_prefix}_labels.npy")



# ---------- 运行 ----------
assert 'train_dataset' in globals()
assert 'val_dataset' in globals()

generate_dataset_to_npy(train_dataset, os.path.join(gen_root, "train"), passes=num_passes, steps=steps)
generate_dataset_to_npy(val_dataset,   os.path.join(gen_root, "test"),  passes=num_passes, steps=steps)

print("✅ All npy generation finished.")



Start generating: total=50000, passes=1, steps=100

=== Pass 1/1 ===


100%|██████████| 196/196 [13:47<00:00,  4.22s/it]



Saved npy dataset:
checkpoints_finetune_cifar100/generated_cifar100_steps_100/train_generated.npy
checkpoints_finetune_cifar100/generated_cifar100_steps_100/train_real.npy
checkpoints_finetune_cifar100/generated_cifar100_steps_100/train_labels.npy

Start generating: total=10000, passes=1, steps=100

=== Pass 1/1 ===


100%|██████████| 40/40 [02:39<00:00,  3.99s/it]



Saved npy dataset:
checkpoints_finetune_cifar100/generated_cifar100_steps_100/test_generated.npy
checkpoints_finetune_cifar100/generated_cifar100_steps_100/test_real.npy
checkpoints_finetune_cifar100/generated_cifar100_steps_100/test_labels.npy
✅ All npy generation finished.


In [10]:
import os
import json
import torch
import numpy as np
from tqdm import tqdm
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from torchvision import transforms, models
import torch.nn.functional as F
from scipy.linalg import sqrtm

# ========= 配置 =========
gen_dir = f"checkpoints_pretrain_imagenet/generated_cifar100_steps_200"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128

# ========= 加载 .npy 数据 =========
print("📥 加载 NPY 数据 ...")
gen_np = np.load(os.path.join(gen_dir, "test_generated.npy"))  # (passes, N, H, W, 3)
real_np = np.load(os.path.join(gen_dir, "test_real.npy"))      # (passes, N, H, W, 3)
label_np = np.load(os.path.join(gen_dir, "test_labels.npy"))   # (passes, N)

# 只取 pass=0（通常你只生成一次）
gen_np = gen_np[0]     # (N, 32, 32, 3)
real_np = real_np[0]

N = gen_np.shape[0]
print(f"Loaded: N = {N}")

# ========= preprocess: 转成张量 =========
to_tensor_3ch = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor()
])
to_gray = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])


# ========= InceptionV3 特征提取器 =========
inception = models.inception_v3(pretrained=True, transform_input=False).to(device)
inception.eval()

class FeatureExtractor(torch.nn.Module):
    def __init__(self, inception_model):
        super().__init__()
        self.inception = inception_model
    def forward(self, x):
        x = F.interpolate(x, size=(299,299), mode='bilinear', align_corners=False)
        x = self.inception.Conv2d_1a_3x3(x)
        x = self.inception.Conv2d_2a_3x3(x)
        x = self.inception.Conv2d_2b_3x3(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = self.inception.Conv2d_3b_1x1(x)
        x = self.inception.Conv2d_4a_3x3(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = self.inception.Mixed_5b(x)
        x = self.inception.Mixed_5c(x)
        x = self.inception.Mixed_5d(x)
        x = self.inception.Mixed_6a(x)
        x = self.inception.Mixed_6b(x)
        x = self.inception.Mixed_6c(x)
        x = self.inception.Mixed_6d(x)
        x = self.inception.Mixed_6e(x)
        x = self.inception.Mixed_7a(x)
        x = self.inception.Mixed_7b(x)
        x = self.inception.Mixed_7c(x)
        x = F.adaptive_avg_pool2d(x, (1,1))
        return x.view(x.size(0), -1)

feat_extractor = FeatureExtractor(inception).to(device).eval()

# ========= 辅助函数：批处理特征提取 =========
def extract_feats_from_np_images(np_images):
    """
    np_images: (N, H, W, 3), float32, [0,1]
    """
    feats_list = []

    for start in tqdm(range(0, len(np_images), batch_size)):
        batch = np_images[start:start + batch_size]

        # NHWC → NCHW
        batch_tensor = torch.tensor(batch).permute(0, 3, 1, 2).float()

        # resize to 299x299
        batch_tensor = F.interpolate(batch_tensor, size=(299, 299),
                                     mode='bilinear', align_corners=False)

        batch_tensor = batch_tensor.to(device)

        with torch.no_grad():
            feats = feat_extractor(batch_tensor)

        feats_list.append(feats.cpu())

    return torch.cat(feats_list, dim=0)


# ========= FID 特征提取 =========
print("🧮 提取生成图特征...")
gen_feats = extract_feats_from_np_images(gen_np)

print("🧮 提取真实图特征...")
real_feats = extract_feats_from_np_images(real_np)

# ========= 计算 FID =========
def calculate_fid_cpu(feats_real, feats_fake):
    feats_real = feats_real.numpy()
    feats_fake = feats_fake.numpy()

    mu_real = feats_real.mean(axis=0)
    mu_fake = feats_fake.mean(axis=0)

    sigma_real = np.cov(feats_real, rowvar=False)
    sigma_fake = np.cov(feats_fake, rowvar=False)

    covmean = sqrtm(sigma_real @ sigma_fake)
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    diff = mu_real - mu_fake
    fid = diff.dot(diff) + np.trace(sigma_real + sigma_fake - 2 * covmean)

    return fid

fid_score = calculate_fid_cpu(real_feats, gen_feats)
print(f"✅ FID = {fid_score:.4f}")

# ========= SSIM =========
print("🧮 计算 SSIM ...")
ssim_scores = []
for i in tqdm(range(N)):
    real_img = real_np[i]     # (H,W,3)
    gen_img  = gen_np[i]

    # 转灰度 & resize
    real_gray = np.dot(real_img, [0.2989, 0.5870, 0.1140])  # RGB → Gray
    gen_gray  = np.dot(gen_img,  [0.2989, 0.5870, 0.1140])

    real_gray = torch.tensor(real_gray).unsqueeze(0)  # 1,H,W
    gen_gray  = torch.tensor(gen_gray).unsqueeze(0)

    real_gray = F.interpolate(real_gray.unsqueeze(0), size=(28,28), mode='bilinear').squeeze().numpy()
    gen_gray  = F.interpolate(gen_gray.unsqueeze(0), size=(28,28), mode='bilinear').squeeze().numpy()

    ssim_val = ssim(real_gray, gen_gray, data_range=1.0)
    ssim_scores.append(ssim_val)

ssim_mean = np.mean(ssim_scores)
print(f"✅ 平均 SSIM = {ssim_mean:.4f}")


📥 加载 NPY 数据 ...
Loaded: N = 10000
🧮 提取生成图特征...


100%|██████████| 79/79 [00:19<00:00,  3.96it/s]


🧮 提取真实图特征...


100%|██████████| 79/79 [00:19<00:00,  4.14it/s]


✅ FID = 46.3038
🧮 计算 SSIM ...


100%|██████████| 10000/10000 [00:07<00:00, 1251.82it/s]

✅ 平均 SSIM = 0.6948


: 

## 微调模型 200步
✅ FID = 43.0414
✅ 平均 SSIM = 0.6992
## 微调模型 100步
✅ FID = 21.8637
✅ 平均 SSIM = 0.8598
## 预训练模型 200步
✅ FID = 46.3038
✅ 平均 SSIM = 0.6948
## 预训练模型 100步
✅ FID = 23.6213
✅ 平均 SSIM = 0.8575